# REGRESSION MODELS WITH KERAS

## Introducción


A pesar de la popularidad de otras librerías más potentes como Pytorch o TensorFlow, estas no son fáciles de usar ni tienen una suave curva de aprendizaje. Por eso, para gente que esta empezando en el deep learning (como yo), la mejor opción es usar Keras. 
Keras es una API de alto nivel para construir modelos de deep learning. Es de sintaxis fácil lo que facilita un rápido desarrollo. Vamos a ver como con Keras, construir una red neuronal compleja nos puede costar tan solo unas pocas líneas de código. 

## Objetivos
* Como usar la libería Keras para construir un modelo de regresión.
* Descargar y limpiar un dataset
* Construir una red neuronal   
* Entrenarla y testearla   



#### Para usar Keras tenemos que installar un framework de Backend
(Si instalamos TensorFlow 2.16 o por encima, se instalará Keras por defecto)




In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
#Importación de liberías y módulos
import pandas as pd
import numpy as np
import keras

import warnings
warnings.simplefilter('ignore', FutureWarning)

c:\Users\ronge\Desktop\PROGRAMACION\DATA SCIENCE\Teoría ML\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


El data set que usaremos es sobre la fuerza compresiva de diferentes muestras de concreto basadas en el volumen de diferentes ingredientes que usamos para hacerlos:



* Cement
* Blast furnace slag
* Fly ash
* Water
* Superplasticizer
* Coarse aggregate
* Fine aggregate


## Descargar y limpiar el Data set


In [3]:
filepath='https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv'
concrete_data = pd.read_csv(filepath)

concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Aquí vamos una imagen general del data set: 

In [ ]:
concrete_data.shape #su forma (filas y columnas)

(1030, 9)

Obtenemos una descripción general y comprobamos si hay valores nulos: 


In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

(Como hemos cogido un data set prefabricado es normal que esté bastante limpio ya)


#### Separación de los datos en predictors y target


La variable target será la fuerza de compresión y los predictores las otras columnas. 


In [7]:
concrete_data_columns = concrete_data.columns

In [8]:
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] #todas las columnas menos Strengh
target = concrete_data['Strength'] # La columna Strengh

<a id="item2"></a>


Breve comprobación de que hemos separado bien: 


In [9]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [10]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64


Normalización de los datos quitando la media y dividiendo por la desviación standard: 

In [11]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069



(Vamos a guardar el número de predictores en n_cols porque luego lo necesitaremos para construir nuestra red)

In [12]:
n_cols = predictors_norm.shape[1] # number of predictors

<a id="item1"></a>


In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

## Construir una red neuronal



Vamos a definir una función que construya nuestro modelo de regresión para que luego podeamos llamarla: 

In [14]:

def regression_model():
    # crear el modelo
    model = Sequential()
    model.add(Input(shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compilar el modelo
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


La función crea un modelo con dos hidden layers, cada uno con 50 nodos.

## Entrenar y testear la red


Creación del modelo llamando a la función


In [15]:

model = regression_model()


Entrenamos y testeamos el modelo al mismo tiempo usando el método fit. Dejamos el 30% de los datos para la validación y entrenamos el modelo para 100 épocas.

In [16]:
# fit the model
model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

Epoch 1/100
23/23 - 1s - 31ms/step - loss: 1569.3243 - val_loss: 1055.7100
Epoch 2/100
23/23 - 0s - 5ms/step - loss: 1361.3019 - val_loss: 864.7631
Epoch 3/100
23/23 - 0s - 4ms/step - loss: 1040.5513 - val_loss: 610.1065
Epoch 4/100
23/23 - 0s - 4ms/step - loss: 655.7473 - val_loss: 363.7569
Epoch 5/100
23/23 - 0s - 4ms/step - loss: 357.4196 - val_loss: 217.6292
Epoch 6/100
23/23 - 0s - 4ms/step - loss: 243.9511 - val_loss: 179.4260
Epoch 7/100
23/23 - 0s - 4ms/step - loss: 223.6265 - val_loss: 173.9593
Epoch 8/100
23/23 - 0s - 4ms/step - loss: 210.8744 - val_loss: 174.8259
Epoch 9/100
23/23 - 0s - 7ms/step - loss: 199.6064 - val_loss: 174.4940
Epoch 10/100
23/23 - 0s - 4ms/step - loss: 192.4017 - val_loss: 171.1288
Epoch 11/100
23/23 - 0s - 4ms/step - loss: 186.1325 - val_loss: 172.3079
Epoch 12/100
23/23 - 0s - 4ms/step - loss: 180.1376 - val_loss: 169.5235
Epoch 13/100
23/23 - 0s - 4ms/step - loss: 175.3194 - val_loss: 168.2798
Epoch 14/100
23/23 - 0s - 4ms/step - loss: 171.2953 - v

## Ejercicio práctico 1



Ahora usando el mismo dataset, vamos a intentar crear un modelo de regresión lineal con 5 hidden layers, cada uno con 50 nodes y ReLU como función de activación, un único output layer y todo optimizado con el Adam. 

In [19]:
def regression_model():
    # crear el modelo
    model = Sequential()
    model.add(Input(shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compilar el modelo
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model



## Ejercicio práctico 2



Ahora entrenamos y evaluamos el modelo reservando únicamente 10% de datos para la validación.

In [20]:
model = regression_model()

model.fit(predictors_norm, target, validation_split=0.1, epochs=100, verbose=2)



Epoch 1/100
29/29 - 1s - 45ms/step - loss: 1495.8284 - val_loss: 878.6647
Epoch 2/100
29/29 - 0s - 4ms/step - loss: 587.7454 - val_loss: 195.2017
Epoch 3/100
29/29 - 0s - 4ms/step - loss: 226.0130 - val_loss: 183.8313
Epoch 4/100
29/29 - 0s - 4ms/step - loss: 196.2491 - val_loss: 168.0079
Epoch 5/100
29/29 - 0s - 4ms/step - loss: 173.3792 - val_loss: 157.4657
Epoch 6/100
29/29 - 0s - 4ms/step - loss: 156.0969 - val_loss: 146.6268
Epoch 7/100
29/29 - 0s - 4ms/step - loss: 138.6860 - val_loss: 128.2925
Epoch 8/100
29/29 - 0s - 4ms/step - loss: 123.3256 - val_loss: 117.7967
Epoch 9/100
29/29 - 0s - 3ms/step - loss: 107.6936 - val_loss: 98.3768
Epoch 10/100
29/29 - 0s - 4ms/step - loss: 94.8238 - val_loss: 91.1448
Epoch 11/100
29/29 - 0s - 4ms/step - loss: 84.4751 - val_loss: 85.5714
Epoch 12/100
29/29 - 0s - 4ms/step - loss: 76.5931 - val_loss: 78.6386
Epoch 13/100
29/29 - 0s - 3ms/step - loss: 66.8098 - val_loss: 70.8301
Epoch 14/100
29/29 - 0s - 3ms/step - loss: 59.4086 - val_loss: 70.6


Basándonos en los resultados, notamos que: 
- Añadir más hidden layers al modelo aumenta su capacidad para aprender y representar relaciones complejas entre los datos. 
- Reducir la propoción de datos seleccionados para la validación supone aumentar la cantidad de datos para el entrenamiento, lo que permite al modelo acceso a más ejemplos de los que aprender. 